# 


In [1]:
from gensim.models import KeyedVectors
from huggingface_hub import hf_hub_download

In [2]:
import pandas as pd
import numpy as np
import torch
import re

In [3]:
import gensim.downloader as api
from gensim.models import KeyedVectors

In [4]:
# Load pre-trained Word2Vec model from gensim-data
model = api.load('word2vec-google-news-300')

# Alternatively, load model from local file if downloaded
# model = KeyedVectors.load_word2vec_format('path/to/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
tbl_labels = pd.read_csv("data/unique_id.txt", delimiter="\\", header=None)
tbl_labels["label_id"] = np.arange(1, tbl_labels.shape[0]+1)
tbl_labels.columns = ["label", "label_id"]
new_order = ["label_id", "label"]
tbl_labels = tbl_labels[new_order]

In [6]:
e1 = model["woman"]
e2 = model["man"]
e3 = model["forest"]

In [7]:
pos_sim = np.sum(e1 * e2)
neg_sim = np.sum(e1 * e3)
neg_sim2 = np.sum(e2 * e3)

In [8]:
pos_sim

4.702738

In [9]:
neg_sim

0.56843174

In [10]:
neg_sim2

0.48191285

In [11]:
table_lookup = pd.DataFrame({"label": [
    "go-kart", "axe", "backscratcher", "first-aid_kit", "hot-air_balloon",
    "hot-water_bottle", "ice-cream_cone", "iceskate", "ping-pong_table",
    "pom-pom", "t-shirt", "ticktacktoe", "yo-yo"
], "updated_label": [
    "small_car", "axes", "backscratchers", "first_aid_kit", "hot_air_balloon",
    "hot_water_bottle", "ice_cream_cones", "ice_skating", "pingpong_table",
    "pom_pom", "tshirt", "tic_tac_toe", "yo_yo"
    ]})
table_lookup.label = table_lookup.label.astype(object)
tbl_labels.label = tbl_labels.label.astype(object)

In [12]:
table_lookup

,label,updated_label
0,go-kart,small_car
1,axe,axes
2,backscratcher,backscratchers
3,first-aid_kit,first_aid_kit
4,hot-air_balloon,hot_air_balloon
5,hot-water_bottle,hot_water_bottle
6,ice-cream_cone,ice_cream_cones
7,iceskate,ice_skating
8,ping-pong_table,pingpong_table
9,pom-pom,pom_pom


In [13]:
tbl_labels_new = pd.merge(left=tbl_labels, right=table_lookup, on="label", how="left")

In [14]:
# Function to find words starting with a given character
def words_starting_with(char):
    vocabulary = list(model.index_to_key)
    words = [word for word in vocabulary if word.startswith(char)]
    return sorted(words)  # Sort the words alphabetically
    
# Example: Find all words starting with 'a'
char = 'asdfasd'
words_with_char = words_starting_with(char)

# Display the words
print(words_with_char)

[]


In [18]:
l_embeddings = []
for prompt, prompt_updated in zip(tbl_labels_new["label"], tbl_labels_new["updated_label"]):
    # use updated prompt if available
    if not pd.isna(prompt_updated):
        prompt = prompt_updated
    # otherwise, use prompt from list
    if prompt in model:
        word_vector = model[prompt]
        l_embeddings.append(word_vector)
    # remove digits if word not found
    elif re.search(r'\d$', prompt):
        m = re.search(r'\d$', prompt)
        prompt = prompt[0:(m.endpos-1)]
        if prompt in model:
            word_vector = model[prompt]
            l_embeddings.append(word_vector)
        else:
            print(f"'{prompt}' not found in the model's vocabulary after removing digits.")
    elif re.search(r'^([a-z]+)(?:_[a-z]+)*$', prompt):
        #m = re.search(r'^([a-z]*)_([a-z]*)$', prompt)
        m = prompt.split('_')
        l_subwords = []
        for subprompt in m:
            subword_vector = model[subprompt]
            l_subwords.append(subword_vector)
        word_vector = np.mean(np.array(l_subwords), axis=0)
        l_embeddings.append(word_vector)
    else:
        print(f"'{prompt}' not found in the model's vocabulary.")

In [19]:
len(l_embeddings)

1854

In [209]:
array_embeddings = np.array(l_embeddings)
# Save the array to a file
np.save('data/word2vec-embeddings.npy', array_embeddings)

# GloVe

In [1]:
from gensim.models import KeyedVectors
import urllib.request
import zipfile
import os

In [2]:
glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
glove_zip_file = "glove.6B.zip"

# Download the GloVe embeddings
urllib.request.urlretrieve(glove_url, glove_zip_file)

# Extract the zip file
with zipfile.ZipFile(glove_zip_file, "r") as zip_ref:
    zip_ref.extractall(".")

# Optional: remove the zip file after extraction
os.remove(glove_zip_file)

KeyboardInterrupt: 

In [3]:
# Load pre-trained GloVe embeddings
glove_file = 'glove.6B.300d.txt'
model = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)

In [4]:
# Get the embeddings for specific words
acorn_embedding = model['acorn']
almond_embedding = model['almond']
airplane_embedding = model['airplane']

# Calculate cosine similarity
similarity_acorn_almond = model.similarity('acorn', 'almond')
similarity_acorn_airplane = model.similarity('acorn', 'airplane')
similarity_almond_airplane = model.similarity('almond', 'airplane')

print(f"Similarity between 'acorn' and 'almond': {similarity_acorn_almond}")
print(f"Similarity between 'acorn' and 'airplane': {similarity_acorn_airplane}")
print(f"Similarity between 'almond' and 'airplane': {similarity_almond_airplane}")

Similarity between 'acorn' and 'almond': 0.140684112906456
Similarity between 'acorn' and 'airplane': -0.05011545866727829
Similarity between 'almond' and 'airplane': 0.006268810480833054


In [5]:
model["air_mattress"]

KeyError: "Key 'air_mattress' not present"

# Fasttext